In [1]:
%matplotlib inline
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
MODEL_SIZE = {
    'Phi3': 4.2,
    'cogvlm2': 19,
    'deepseekvl': 7,
    'internvl2': 8,
    'llava1.6': 7,
    'molmo': 7,
    'pixtral2': 12,
    'qwen2vl': 7,
    'smolvlm': 2,
}

PRETTY_NAMES = {
    'BING_GPT-4': 'Bing w/ GPT4',
    'Phi3': 'Phi-3.5',
    'cogvlm2': 'CogVLM2',
    'deepseekvl': 'Deepseek-vl',
    'internvl2': 'InternVL2',
    'llava1.6': 'Llava-1.6',
    'molmo': 'Molmo',
    'pixtral2': 'Pixtral',
    'qwen2vl': 'Qwen2-VL',
    'smolvlm': 'SmolVLM',
}

In [3]:
sns.set_theme(style="darkgrid")

In [4]:
def plot_accuracies(data, key, csv_output_path, output_path):
    perfs = data.groupby(by=['Algorithm', 'Dataset'], as_index=False).mean()
    perfs.to_csv(csv_output_path)
    
    fig = plt.figure(figsize=(15, 5))
    sns.barplot(x='Algorithm', y=key, hue='Dataset', errorbar=None, data=perfs)
    plt.xticks(rotation=30)
    plt.ylim(0.0, 1.0)
    plt.legend(ncol=3)
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close(fig)
    
def plot_picture_accuracies(data, key, csv_output_path, output_path):
    perfs = data.groupby(by=['Picture', key]).size().reset_index(name='Count')
    perfs.to_csv(csv_output_path)
    
    fig = plt.figure(figsize=(7.5, 10))
    sns.barplot(y='Picture', x='Count', hue=key, errorbar=None, data=perfs, orient='h')
    plt.xticks(rotation=30)
    plt.legend(ncol=1)
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close(fig)
    
def load_csv(csv_path):
    data = pd.read_csv(csv_path, sep='|', index_col=0)
    # data = data.drop(data[data['Algoritmo'] == 'BING_GPT-4_eng'].index)
    data.drop(index=data[data['Percorso'] == './dataset/i17_wiki_portici.jpg'].index)
    data['Soggetto_originale'] = data['Soggetto_originale'].apply(lambda x: x.replace('/Portici', ''))
    data['Algorithm'] = data['Algoritmo'].map(lambda x: PRETTY_NAMES[x[:-4]])
    data['Subject Accuracy'] = data['Soggetto_1'] == 'OK'
    data['City Accuracy'] = data['Città_1'] == 'OK'
    data['Subject Accuracy (2Q)'] = (data['Soggetto_1'] == 'OK') | ((data['Soggetto_1'] != 'OK') & (data['Soggetto_2'] == 'OK'))
    data['Picture'] = data['Città'] + ' / ' + data['Soggetto_originale']
    
    return data

In [5]:
def lighten_color(color, amount=0.6):
    # Convert hex color to RGB
    rgb = mcolors.hex2color(color)
    # Increase the brightness by adding the amount
    rgb_lightened = [(1 - amount) * x + amount for x in rgb]
    # Convert back to hex
    return mcolors.rgb2hex(rgb_lightened)


def plot_picture_accuracies_combo(eng_data, ita_data, key, csv_output_path, output_path):
    perfs_eng = eng_data.groupby(by=['Picture', key]).size().reset_index(name='Count')
    perfs_ita = ita_data.groupby(by=['Picture', key]).size().reset_index(name='Count')
    perfs_eng[key] = perfs_eng[key].apply(lambda x: x + ' (EN)')
    perfs_ita[key] = perfs_ita[key].apply(lambda x: x + ' (IT)')
    perfs_combo = pd.concat([perfs_eng, perfs_ita], axis=0)
    perfs_combo.to_csv(csv_output_path)

    color_map = {
        'OK (IT)': lighten_color('#2ca02c'),
        'OK (EN)': '#2ca02c',
        'NO (IT)': lighten_color('#ff7f0e'),
        'NO (EN)': '#ff7f0e',
        'ERR (IT)': lighten_color('#d62728'),
        'ERR (EN)': '#d62728',
    }
    
    fig = plt.figure(figsize=(7.5, 10))
    sns.barplot(y='Picture', x='Count', hue=key, errorbar=None, data=perfs_combo, orient='h', hue_order=['OK (IT)', 'OK (EN)', 'NO (IT)', 'NO (EN)', 'ERR (IT)', 'ERR (EN)'], palette=color_map)
    plt.xticks(rotation=30)
    plt.legend(ncol=1)
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close(fig)

In [6]:
def build_plots(data, suffix):
    plot_accuracies(data, 'Subject Accuracy', f'tables/subject1_{suffix}.csv', f'figures/subject1_{suffix}.pdf')
    plot_accuracies(data, 'City Accuracy', f'tables/city1_{suffix}.csv', f'figures/city1_{suffix}.pdf')
    plot_accuracies(data, 'Subject Accuracy (2Q)', f'tables/subject2_{suffix}.csv', f'figures/subject2_{suffix}.pdf')
    
    plot_picture_accuracies(data, 'Soggetto_1', f'tables/picture_subject1_{suffix}.csv', f'figures/picture_subject1_{suffix}.pdf')
    plot_picture_accuracies(data, 'Città_1', f'tables/picture_city1_{suffix}.csv', f'figures/picture_city1_{suffix}.pdf')

In [7]:
def build_plots_combo(eng_data, ita_data):
    plot_picture_accuracies_combo(eng_data, ita_data, 'Soggetto_1', 'tables/picture_subject1_combo.csv', 'figures/picture_subject1_combo.pdf')
    plot_picture_accuracies_combo(eng_data, ita_data, 'Città_1', 'tables/picture_city1_combo.csv', 'figures/picture_city1_combo.pdf')

In [8]:
def diff_table(data_before, data_after, key_before, key_after, csv_out_path, fig_out_path):
    counts_before = data_before.groupby('Algorithm')[key_before].value_counts()
    counts_after = data_after.groupby('Algorithm')[key_after].value_counts()
    
    df = pd.DataFrame(columns=['OK', 'NO', 'ERR'])
    
    for k in PRETTY_NAMES.values():
        values = []
        int_values = []
        for subset in ['OK', 'NO', 'ERR']:
            values.append(str(counts_after[k].get(subset, 0) - counts_before[k].get(subset, 0)))
            int_values.append(counts_after[k].get(subset, 0) - counts_before[k].get(subset, 0))
        df.loc[k] = int_values
        
    df.to_csv(csv_out_path)
            
    fig, ax = plt.subplots(figsize=(10, 6))

    COLUMNS = ['OK', 'NO', 'ERR']

    df[COLUMNS].plot(kind='bar', ax=ax, color=['#2ca02c', '#ff7f0e', '#d62728'], width=0.3)

    ax.set_xlabel("Models")
    ax.set_ylabel("Difference")
    ax.legend()
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    plt.savefig(fig_out_path)
    plt.close(fig)

In [9]:
eng_data = load_csv('./all_outputs_eng.csv')
ita_data = load_csv('./all_outputs_ita.csv')

In [10]:
build_plots(eng_data, 'eng')
build_plots(ita_data, 'ita')
build_plots_combo(eng_data, ita_data)

In [11]:
diff_table(ita_data, eng_data, 'Soggetto_1', 'Soggetto_1', 'tables/ita_eng_diff.csv', 'figures/ita_eng_diff.pdf')

In [12]:
eng_subj_df = diff_table(eng_data[eng_data['Soggetto_1'] != 'OK'], eng_data[eng_data['Soggetto_1'] != 'OK'], 'Soggetto_1', 'Soggetto_2', 'tables/eng_subject_diff.csv', 'figures/eng_subject_diff.pdf')
ita_subj_df = diff_table(ita_data[ita_data['Soggetto_1'] != 'OK'], ita_data[ita_data['Soggetto_1'] != 'OK'], 'Soggetto_1', 'Soggetto_2', 'tables/ita_subject_diff.csv', 'figures/ita_subject_diff.pdf')